**Terms**:
- UCB: mathematical decision-making formula for solving "Exploration-Exploitation trade-off"
- UCT: Tree policy used for selection and expansion phase of MCTS. Uses UCB for finding optimal action.
<br>
Reference: Bandit based Monte-Carlo Planning
$$UCT = \frac{w_i}{n_i} +2 C_p \sqrt{\frac{2\ln N_p}{n_i}}$$

In [2]:
import numpy as np
from math import sqrt,log
from collections import defaultdict
from tic_tac_toe import Tictactoe 

In [ ]:
# UCT-MCTS Implementation for tic-tac-toe
C_p = 1 / sqrt(2)  # EXPLORATION_CONSTANT


class MCTSNode:
    def __init__(self, state: Tictactoe, parent=None, parent_action=None):
        self.state = state
        self.parent = parent
        self.parent_action = parent_action
        self.children: list[MCTSNode] = []
        self.visit_counts = 0
        self.results = defaultdict(int)  # store simulation outcome
        # initial root state no simulation
        self.results[1] = 0  # X wins
        self.results[-1] = 0  # X loses
        self.untried_actions = (
            state.get_legal_moves()
        )  # should to fetch all the legals move from the game

    def reward(self):
        wins = self.results[1]
        loses = self.results[-1]
        return wins - loses

    def visits(self):
        return self.visit_counts

    def expand(self):
        action = self.untried_actions.pop()
        next_state = self.state.next_state(action)
        child_node = MCTSNode(next_state, parent=self, parent_action=action)
        self.children.append(child_node)
        return child_node

    def is_terminal_node(self):
        return self.state.is_gameover()

    def simulation(self):
        curr_sim_state = self.state
        while not curr_sim_state.is_gameover():
            possible_moves = curr_sim_state.get_legal_moves()
            action = self.default_policy(possible_moves)
            curr_sim_state = curr_sim_state.next_state(action)

        return curr_sim_state.game_result()

    def default_policy(self, possible_moves):
        random_move_idx = np.random.randint(len(possible_moves))
        return possible_moves[random_move_idx]

    def backpropagate(self, result):
        self.visit_counts += 1
        self.results[result] += 1
        if self.parent:
            self.parent.backpropagate(result)

    def is_fully_expanded(self):
        return len(self.untried_actions) == 0

    def best_child(self):
        uct_scores = [self.calculate_uct_score(child) for child in self.children]
        best_score = np.argmax(uct_scores)
        return self.children[best_score]

    def calculate_uct_score(self, child):
        w = child.reward()
        n = child.visits()
        N = child.parent.visits()
        if n == 0:
            return float("inf")
        exploitation = w / n  # avg reward
        exploration = 2 * C_p * sqrt(2 * log(N) / n)
        uct_score = exploitation + exploration
        return uct_score

    def tree_policy(self):
        current_node = self
        while not current_node.is_terminal_node():
            if not self.is_fully_expanded():
                return current_node.expand()
            else:
                current_node = current_node.best_child()
        return current_node

    def best_action(self, simulations_number=100):
        for _ in range(simulations_number):
            v = self.tree_policy()
            reward = v.simulation()
            v.backpropagate(reward)
        best_child = self.best_child()
        return best_child.parent_action


In [20]:
game = Tictactoe()
game.print_board()

_|_|_
_|_|_
_|_|_


In [21]:
next_state = game.next_state(2)

In [22]:
next_state.print_board()

_|_|X
_|_|_
_|_|_


In [ ]:
#root = MCTSNode(state=initial_state)
#action = root.best_action(simulations_number=100)

In [27]:
# mctsnode = MCTSNode([1,2,3])

In [18]:
# 9X9 grid, will have 9 (3X3) sub grid, winner must win 3 row or in diagonal to win the game
class TicTacToe:
    pass

In [ ]:
import random

game = Tictactoe()


def get_random_move():
    return random.randint(0, game.SIZE**2 - 1)


play_ai = True
player = random.randint(0, 1)

if player == 0:
    player = game.X
    ai = game.O
else:
    player = game.O
    ai = game.X

while not (game.is_gameover()):
    print("game turn:", game.turn)
    if game.turn == player:
        # Player turn
        if play_ai:
            move = get_random_move()
            while not game.is_valid_move(move):
                move = get_random_move()

        else:
            move = get_random_move()
            while not game.is_valid_move(move):
                print("Invalid input")
                move = int(input())

        game.make_move(move)
    else:
        # MCTS AI turn
        root = MCTSNode(state=game)
        ai_move = root.best_action(simulations_number=100)
        print("ai moveeeeeeee", ai_move)
        game.make_move(ai_move)


##    else:
#        # AI turn
#        # feed all the board status to NN
#        net_output = neuralnet.feed_forward(game.export_board())
#
#        movelist = get_movelist(net_output)
#        ai_move = get_move(game, movelist)
#
#        game.make_move(ai_move)

print("\n\nFinal Board:")
game.print_board()

if game.is_gameover():
    print(str(game.winner) + " won!")
else:
    print("Draw!")

game turn: X


ValueError: attempt to get argmax of an empty sequence